# Update Buyer Seller List

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import seaborn as sns
import datetime

In [2]:
buyers=pd.read_csv('retailer_buyer_list.csv')

In [3]:
bid_ytd=pd.read_csv('bid_ytd.csv')
stock_ytd=pd.read_csv('stock_ytd.csv')

In [4]:
stock_ytd['submitted_on']=pd.to_datetime(stock_ytd['submitted_on'])
stock_ytd['buyer']=stock_ytd['buyer'].str.lower()

In [5]:
start=datetime.datetime(2021,4,1)

In [6]:
apr_may_stock=stock_ytd[(stock_ytd['submitted_on']>=start)]
may_june_stock=stock_ytd[stock_ytd['submitted_on'].isnull()]

In [7]:
up_stock=apr_may_stock.merge(may_june_stock, how='outer').reset_index(drop=True)

In [8]:
bid_ytd['bid_placed_on']=pd.to_datetime(bid_ytd['bid_placed_on'])

up_bid=bid_ytd[bid_ytd['bid_placed_on']>=start].reset_index(drop=True)
up_bid['buyer']=up_bid['buyer'].str.lower()

In [9]:
may_june_stock.shape

(1327, 13)

In [10]:
up_bid.shape

(24813, 18)

## Auctions

In [11]:
end=datetime.datetime(2021,5,1)

In [12]:
up_win=up_bid[up_bid['bid']>=up_bid['winning_price']]
auc_win=(up_win[up_win['bid_placed_on']<end].groupby(['buyer'])['winning_price']
         .sum()
         .rename('auction_gmv')
         .sort_values(ascending=False)
         .reset_index())

auc_win.sort_values(by='auction_gmv',ascending=False).head()

,buyer,auction_gmv
0,rue gilt,608464.0
1,harley 2019,201513.0
2,bagriculture,132739.0
3,season 2 consign,27858.0
4,bag borrow steal,23600.0


In [13]:
up_win[up_win['bid_placed_on']<end]['winning_price'].sum()

1148637.0

In [14]:
auc_win['auction_gmv'].sum()

1148637.0

## Stock

In [15]:
# new=up_stock[(up_stock['submitted_on']>=end)].merge(up_stock[(up_stock['submitted_on'].isnull())], how='outer')

In [16]:
grp_stock=(up_stock[(up_stock['submitted_on']<end)].groupby(['buyer'])['price']
         .sum()
         .rename('stock_gmv')
         .sort_values(ascending=False)
         .reset_index())
grp_stock.sort_values(by='stock_gmv',ascending=False).head()

,buyer,stock_gmv
0,everything but the house,248655.0
1,the realreal,154131.0
2,rue gilt,142874.0
3,season 2 consign,107756.0
4,the luxury bee,70652.0


In [17]:
up_stock[(up_stock['submitted_on']<end)]['price'].sum()

1231542.0

In [18]:
grp_stock['stock_gmv'].sum()

1231542.0

## Combine

In [19]:
q1_gmv=grp_stock.merge(auc_win, how='outer')

q1_gmv=q1_gmv.fillna(0)

q1_gmv['total_gmv']=q1_gmv['auction_gmv']+q1_gmv['stock_gmv']

q1_gmv.head()

,buyer,stock_gmv,auction_gmv,total_gmv
0,everything but the house,248655.0,0.0,248655.0
1,the realreal,154131.0,0.0,154131.0
2,rue gilt,142874.0,608464.0,751338.0
3,season 2 consign,107756.0,27858.0,135614.0
4,the luxury bee,70652.0,0.0,70652.0


In [20]:
all_buy=(buyers[['User ID', 'Status','Retailer Name']]
         .rename(columns={'User ID': 'user_id', 'Retailer Name': 'buyer'}))
all_buy['buyer']=all_buy['buyer'].str.lower()

all_buy.head()

,user_id,Status,buyer
0,186104,Active,sydney.s
1,182715,Active,wgaca
2,205200,Active,jonathanriss
3,2829,Active,chic girl
4,188669,Active,bag borrow steal


In [21]:
update=all_buy.merge(q1_gmv, how='outer')
update.loc[:,'Status']='Active'

update.loc[update['total_gmv'].isnull(), 'Status']='Inactive'

update.head()

,user_id,Status,buyer,stock_gmv,auction_gmv,total_gmv
0,186104.0,Active,sydney.s,4040.0,1426.0,5466.0
1,182715.0,Active,wgaca,0.0,720.0,720.0
2,205200.0,Inactive,jonathanriss,NaN,NaN,NaN
3,2829.0,Active,chic girl,8950.0,1473.0,10423.0
4,188669.0,Active,bag borrow steal,29477.0,23600.0,53077.0


In [22]:
update['total_gmv'].sum()

2380179.0

In [23]:
update=update.fillna(0)

In [24]:
# update.to_excel('updated_gmv_april.xlsx')

## Status Update

In [37]:
bs=pd.read_csv('bs.csv')

In [38]:
bs['Status']='Active'
bs.loc[(bs['April']==0) & (bs['May']==0), 'Status']='Inactive'

In [39]:
bs.head()

,User ID,Status,Sign up year,Location,April,May
0,186104,Active,2017,CA,5466,0.0
1,182715,Active,2017,FL,720,69103.0
2,205200,Inactive,2018,NY,0,0.0
3,2829,Active,2018,FL,10507,2774.0
4,188669,Active,2018,PA,43711,124503.0


In [41]:
# bs.to_excel('buy_status_update.xlsx')

In [49]:
sup_stat=pd.read_csv('sup_stat.csv')

In [50]:
sup_stat['Status']='Active'
sup_stat.loc[(sup_stat['Apr_Listed']==0) & (sup_stat['May_Listed']==0), 'Status']='Inactive'
sup_stat.head()

,Seller ID,Status,Note,Location,April 2021 GMV,May 2021 GMV,Supplier,Apr_Listed,Apr_Sold,May_Listed,May_Sold
0,204684,Active,NaN,Japan,"$108,407.00","$109,265.00",eLady,1568.0,185.0,5214.0,67.0
1,202899,Active,NaN,Japan,"$85,872.00","$91,999.00",BrandOff,1300.0,128.0,1696.0,88.0
2,213070,Active,NaN,NY,"$27,265.00","$8,365.00",bagriculture,929.0,55.0,1352.0,10.0
3,198548,Active,NaN,Japan,"$133,140.00","$121,387.00",Next Innovation,2629.0,337.0,2487.0,288.0
4,196448,Active,NaN,Japan,"$507,464.00","$540,275.00",aucnet,65193.0,706.0,51861.0,260.0


In [53]:
# sup_stat.to_excel('sup_status_update.xlsx')